<a href="https://colab.research.google.com/github/GianFadiga/MCDA_FS/blob/main/mcda_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MCDA: PROJECT - Final Algorithm

In [38]:
import pandas as pd
import plotly.express as px
import numpy as np

class DataAnalyzer:
    def __init__(self, file_path):
        self.file_path = file_path
        self.df = None
        self.pesos = None
        self.tipos_dados = None
        self.proporcionalidade = None
        self.valores_bom = None
        self.valores_neutro = None
        self.calculo_df = None
        # Adicione o mapeamento de string aqui para ser acessível
        self.string_columns_map = {}


    def load_and_prepare_data(self):
        """Carrega e prepara os dados de forma genérica para strings"""
        self.df = pd.read_csv(self.file_path, encoding='UTF-8', sep=',', skipinitialspace=True)

        # Remove colunas totalmente vazias
        self.df = self.df.dropna(axis=1, how='all')

        # Extrai configurações ANTES de mapear colunas string
        self.pesos = self.df.iloc[0].dropna().astype(float)
        self.tipos_dados = self.df.iloc[1].dropna()
        self.proporcionalidade = self.df.iloc[2].dropna()

        # Identifica automaticamente colunas string e seus pts_string
        # É importante fazer isso *depois* de ler os tipos_dados
        self._mapear_colunas_string()

        # Extrai BOM/NEUTRO
        # Usar .get para evitar KeyError se 'Modelo' não for a primeira coluna
        bom_row = self.df[self.df.iloc[:, 0] == 'BOM']
        neutro_row = self.df[self.df.iloc[:, 0] == 'NEUTRO']

        if not bom_row.empty:
             self.valores_bom = bom_row.iloc[0].dropna()
        else:
             print("Aviso: Linha 'BOM' não encontrada.")
             self.valores_bom = pd.Series(dtype='object')

        if not neutro_row.empty:
             self.valores_neutro = neutro_row.iloc[0].dropna()
        else:
             print("Aviso: Linha 'NEUTRO' não encontrada.")
             self.valores_neutro = pd.Series(dtype='object')


        # Cria calculo_df pulando as 5 primeiras linhas (pesos, tipo, prop, BOM, NEUTRO)
        # Assume que 'Modelo' é a primeira coluna para a lógica BOM/NEUTRO
        self.calculo_df = self.df.iloc[5:].reset_index(drop=True)
        self._convert_data_types()

    def _convert_data_types(self):
        """Converte os tipos de dados conforme especificado"""
        # Garante que valores_bom e valores_neutro sejam Series
        if self.valores_bom is None: self.valores_bom = pd.Series(dtype='object')
        if self.valores_neutro is None: self.valores_neutro = pd.Series(dtype='object')

        for col in self.tipos_dados.index:
            if col not in self.calculo_df.columns:
                print(f"Aviso: Coluna '{col}' definida nos tipos mas não encontrada nos dados para cálculo.")
                continue

            tipo = self.tipos_dados.get(col) # Usar .get para segurança

            if tipo == 'number':
                # Tenta converter BOM/NEUTRO para float, usa NaN se falhar
                self.valores_bom[col] = pd.to_numeric(self.valores_bom.get(col), errors='coerce')
                self.valores_neutro[col] = pd.to_numeric(self.valores_neutro.get(col), errors='coerce')
                # Converte coluna de dados
                self.calculo_df[col] = pd.to_numeric(self.calculo_df[col], errors='coerce')

            elif tipo == 'boolean':
                 # --- CORREÇÃO BOOLEAN ---
                 # Converte BOM/NEUTRO: 'TRUE' -> True, outros -> False
                 self.valores_bom[col] = str(self.valores_bom.get(col, '')).strip().upper() == 'TRUE'
                 self.valores_neutro[col] = str(self.valores_neutro.get(col, '')).strip().upper() == 'TRUE'
                 # Converte coluna de dados com mapeamento robusto
                 self.calculo_df[col] = self.calculo_df[col].astype(str).str.strip().str.upper().map({'TRUE': True, 'FALSE': False})
                 # Define um padrão para valores não mapeados (ex: NaN ou outros textos)
                 self.calculo_df[col] = self.calculo_df[col].fillna(False).astype(bool) # Trata NaN e força tipo bool

            elif tipo == 'string':
                # Apenas garante que a coluna de dados seja string
                self.calculo_df[col] = self.calculo_df[col].astype(str)

            # Lidar com colunas pts_string (não precisam de conversão aqui)
            elif tipo == 'pts_string':
                 continue # Ignora pts_string na conversão principal

            else:
                 print(f"Aviso: Tipo de dado desconhecido '{tipo}' para coluna '{col}'.")


    def calcular_pontuacao_numerica(self, valor, bom, neutro, peso, proporcionalidade_tipo):
        """Calcula pontuação para dados numéricos e booleanos"""

        # --- CORREÇÃO BOOLEAN LOGIC ---
        # Trata explicitamente valores booleanos primeiro
        # Usar isinstance com np.bool_ também por causa do Pandas/Numpy
        if isinstance(valor, (bool, np.bool_)):
            # Caso especial: BOM e NEUTRO são iguais (ambos True ou ambos False)
            if bom == neutro:
                return peso if valor == bom else 0 # Pontua se igual ao único alvo, senão 0
            # Caso normal: BOM e NEUTRO são diferentes
            else:
                if valor == bom:
                    return peso  # Acertou o BOM
                elif valor == neutro:
                    return 0     # Acertou o NEUTRO
                else:
                    # Não é nem BOM nem NEUTRO (deve ser o oposto do BOM e do NEUTRO)
                    return -peso # Penalidade

        # Se não for booleano, segue para a lógica numérica
        # Verifica se algum valor essencial é NaN antes de prosseguir
        if pd.isna(valor) or pd.isna(bom) or pd.isna(neutro) or pd.isna(peso):
             return 0 # Não pontua se faltar informação

        # Se BOM == NEUTRO para números, retorna 0 (ou peso se valor for igual?)
        # Mantendo a lógica original: retorna 0 se bom == neutro
        if bom == neutro:
            return 0

        # Tenta converter para float (já deveriam ser, mas por segurança)
        try:
            valor = float(valor)
            bom = float(bom)
            neutro = float(neutro)
            peso = float(peso)
        except (ValueError, TypeError):
            # Se a conversão falhar aqui, algo está errado antes
            print(f"Aviso: Falha na conversão para float em calcular_pontuacao_numerica - Valor: {valor}, Bom: {bom}, Neutro: {neutro}")
            return 0

        # Lógica de proporcionalidade (mantida como estava, mas agora com valores mais seguros)
        # Adicionando tratamento para divisão por zero ou neutro zero
        if proporcionalidade_tipo == 'proportional':
            if bom > neutro: # Caso normal crescente
                 if valor >= bom:
                     return peso
                 elif valor >= neutro:
                      # Evita divisão por zero se bom == neutro (já tratado acima)
                     return ((valor - neutro) / (bom - neutro)) * peso
                 else: # Abaixo do neutro
                     # Evita divisão por zero se neutro for 0 e valor negativo
                     denominador_penalidade = neutro if neutro != 0 else 1 # Usa 1 se neutro for 0
                     return -((abs(neutro - valor)) / abs(denominador_penalidade)) * peso # Penalidade proporcional à distância do neutro
            elif bom < neutro: # Caso decrescente (ex: menor é melhor até BOM) - Incomum para 'proportional' mas possível
                 if valor <= bom:
                     return peso
                 elif valor <= neutro:
                     return ((valor - bom) / (neutro - bom)) * peso # Inverte a lógica
                 else: # Acima do neutro (pior)
                    denominador_penalidade = neutro if neutro != 0 else 1
                    return -((abs(valor - neutro)) / abs(denominador_penalidade)) * peso
            else: # bom == neutro
                 return 0

        elif proporcionalidade_tipo == 'i_proportional':
             if bom < neutro: # Caso normal decrescente (menor é melhor)
                 if valor <= bom:
                     return peso
                 elif valor <= neutro:
                      # Evita divisão por zero se neutro == bom (já tratado acima)
                     return ((neutro - valor) / (neutro - bom)) * peso
                 else: # Acima do neutro (pior)
                     denominador_penalidade = neutro if neutro != 0 else 1
                     return -((abs(valor - neutro)) / abs(denominador_penalidade)) * peso # Penalidade proporcional
             elif bom > neutro: # Caso crescente (maior é melhor até BOM) - Incomum para 'i_proportional'
                 if valor >= bom:
                     return peso
                 elif valor >= neutro:
                     return ((bom - valor) / (bom - neutro)) * peso # Inverte a lógica
                 else: # Abaixo do neutro (pior)
                     denominador_penalidade = neutro if neutro != 0 else 1
                     return -((abs(neutro - valor)) / abs(denominador_penalidade)) * peso
             else: # bom == neutro
                 return 0

        return 0 # Caso tipo de proporcionalidade não seja reconhecido


    def _mapear_colunas_string(self):
        """Mapeia automaticamente colunas string para suas colunas pts_string"""
        self.string_columns_map = {}
        if self.tipos_dados is None: # Garante que tipos_dados foi carregado
             print("Erro: tipos_dados não carregado antes de mapear colunas string.")
             return

        string_cols = [col for col in self.tipos_dados.index
                      if self.tipos_dados[col] == 'string']

        df_cols = list(self.df.columns) # Lista das colunas originais

        for col in string_cols:
            try:
                # Encontra a posição da coluna string
                col_idx = df_cols.index(col)

                # A coluna pts_string deve ser a próxima coluna
                if col_idx + 1 < len(df_cols):
                    pts_potential_col = df_cols[col_idx + 1]
                    # Verifica se a próxima coluna é realmente a de pontos (opcional, mas bom)
                    # Poderia verificar se o tipo dela é 'pts_string' na linha 1, mas vamos confiar na posição por enquanto
                    new_pts_col_name = f"{col},pts_string"
                    self.string_columns_map[col] = new_pts_col_name # Armazena o nome *novo*

                    # Renomeia a coluna no DataFrame original
                    self.df = self.df.rename(columns={pts_potential_col: new_pts_col_name})
                    print(f"Coluna string '{col}' mapeada para '{new_pts_col_name}' (original: '{pts_potential_col}')")

                else:
                    print(f"Aviso: Coluna string '{col}' não tem uma coluna seguinte para ser pts_string.")
            except ValueError:
                 print(f"Aviso: Coluna string '{col}' definida nos tipos não encontrada no CSV.")
            except Exception as e:
                 print(f"Erro ao mapear coluna string '{col}': {e}")


    def calcular_pontuacao_string(self, valor, coluna):
        """Calcula pontuação genérica para qualquer string"""
        # Obtém o nome CORRETO da coluna de pontos, que foi renomeada
        pts_col = self.string_columns_map.get(coluna)

        if not pts_col:
            # Isso não deveria acontecer se _mapear_colunas_string funcionou
            print(f"Aviso interno: Mapeamento para coluna string '{coluna}' não encontrado.")
            return 0

        if pts_col not in self.df.columns:
             # Isso pode acontecer se o renomear falhou ou a coluna não existia
             print(f"Aviso: Coluna de pontos '{pts_col}' esperada para '{coluna}' não existe no DataFrame.")
             return 0

        try:
            # --- CORREÇÃO STRING ---
            # 1. Criar o mapeamento a partir das linhas de dados do df original
            dados_mapa = self.df.iloc[5:].copy()

            # 2. Converter a coluna de pontos para numérico DEPOIS de selecionar os dados
            dados_mapa[pts_col] = pd.to_numeric(dados_mapa[pts_col], errors='coerce')

            # 3. Remover linhas onde a string original OU o ponto convertido são NaN
            dados_mapa = dados_mapa.dropna(subset=[coluna, pts_col])

            # 4. Criar o dicionário de mapeamento (String -> Ponto Numérico)
            mapping = pd.Series(dados_mapa[pts_col].values, index=dados_mapa[coluna]).to_dict()

            # Se o mapping estiver vazio após a limpeza, retorna 0
            if not mapping:
                 print(f"Aviso: Mapeamento de pontuação para '{coluna}' está vazio após limpeza.")
                 return 0

            # Remove espaços e busca o valor no mapeamento
            valor_clean = str(valor).strip()
            # O valor buscado no dicionário JÁ É float (ou int)
            pontuacao_base = mapping.get(valor_clean, 0) # Pega o ponto base (já numérico)

            # Multiplica o ponto base pelo peso da coluna string
            peso_coluna = float(self.pesos.get(coluna, 0)) # Pega o peso com segurança
            resultado = pontuacao_base * peso_coluna

            # Debugging extra (opcional):
            # print(f"  Debug String: Col: {coluna}, Valor: '{valor_clean}', Ponto Base: {pontuacao_base}, Peso: {peso_coluna}, Result: {resultado}")

            return resultado

        except Exception as e:
            # Captura erros mais específicos se possível
            print(f"Erro detalhado ao calcular pontuação para '{coluna}' (Valor: '{valor}'): {type(e).__name__} - {str(e)}")
            # Imprimir o estado do mapeamento pode ajudar
            # print(f"  Debug - Estado do mapping problemático: {mapping}")
            return 0

    def calcular_pontuacoes(self):
        """Calcula todas as pontuações com verificação extra"""
        if self.calculo_df is None or self.tipos_dados is None or self.pesos is None:
             print("Erro: Dados não carregados ou preparados corretamente. Execute load_and_prepare_data() primeiro.")
             return

        print("Colunas disponíveis para cálculo:", list(self.calculo_df.columns))
        print("Pesos aplicáveis:", self.pesos.to_dict())
        print("Tipos de Dados:", self.tipos_dados.to_dict())


        for col in self.pesos.index:
            # Verifica se a coluna de peso existe nos dados de cálculo
            if col not in self.calculo_df.columns:
                # Verifica se é uma coluna string (cujo cálculo usa a coluna original)
                if col not in self.string_columns_map:
                     print(f"  Aviso: Coluna de peso '{col}' não encontrada no DataFrame de cálculo e não é uma coluna string mapeada.")
                     continue
                # Se for string, o cálculo usará a coluna original, então está OK não estar em calculo_df diretamente,
                # mas precisamos garantir que o tipo é 'string'
                elif self.tipos_dados.get(col) != 'string':
                     print(f"  Aviso: Coluna de peso '{col}' não encontrada no DataFrame de cálculo, mas mapeada como string ({self.string_columns_map.get(col)}). Verifique a consistência.")
                     continue
                # Se for string mapeada, o processamento ocorrerá no bloco 'string' abaixo

            print(f"\nProcessando coluna: {col}")

            # Pega informações com segurança usando .get()
            tipo = self.tipos_dados.get(col)
            proporcionalidade = self.proporcionalidade.get(col)
            peso = self.pesos.get(col, 0) # Usa 0 se peso não for encontrado por algum motivo
            valor_bom = self.valores_bom.get(col)
            valor_neutro = self.valores_neutro.get(col)
            pont_col_name = f"{col}_pontuacao" # Nome da coluna de resultado

            try:
                if tipo in ['number', 'boolean']:
                    # Verifica se todas as informações necessárias estão presentes
                    if pd.isna(valor_bom) or pd.isna(valor_neutro):
                         print(f"  Aviso: Valores BOM ({valor_bom}) ou NEUTRO ({valor_neutro}) ausentes/inválidos para '{col}'. Pulando pontuação.")
                         self.calculo_df[pont_col_name] = 0
                         continue
                    if not proporcionalidade:
                         print(f"  Aviso: Proporcionalidade não definida para '{col}'. Pulando pontuação.")
                         self.calculo_df[pont_col_name] = 0
                         continue

                    print(f"  Tipo: {tipo}, Proporcionalidade: {proporcionalidade}")
                    print(f"  Valores BOM: {valor_bom}, NEUTRO: {valor_neutro}, Peso: {peso}")

                    self.calculo_df[pont_col_name] = self.calculo_df.apply(
                        lambda row: self.calcular_pontuacao_numerica(
                            row[col],
                            valor_bom,
                            valor_neutro,
                            peso,
                            proporcionalidade
                        ) if col in row else 0, # Adiciona verificação se col existe na linha
                        axis=1
                    )

                elif tipo == 'string':
                    # Verifica se a coluna original existe
                    if col not in self.calculo_df.columns:
                         print(f"  Erro Interno: Coluna string '{col}' não encontrada em calculo_df para aplicar pontuação.")
                         self.calculo_df[pont_col_name] = 0
                         continue

                    pts_col_esperado = self.string_columns_map.get(col)
                    print(f"  Tipo: {tipo}. Coluna de pontos esperada: {pts_col_esperado}. Peso: {peso}")
                    # A validação da existência de pts_col é feita dentro de calcular_pontuacao_string

                    self.calculo_df[pont_col_name] = self.calculo_df.apply(
                        lambda row: self.calcular_pontuacao_string(row[col], col),
                        axis=1
                    )

                # Imprime resultado parcial se a coluna foi criada
                if pont_col_name in self.calculo_df:
                    print(f"  Resultado parcial (Top 5):\n{self.calculo_df[[col, pont_col_name]].head().to_string(index=False)}")
                else:
                     print(f"  Aviso: Coluna de pontuação '{pont_col_name}' não foi criada.")

            except KeyError as e:
                 print(f"  Erro de Chave (KeyError) ao processar coluna '{col}': {e}. Verifique se a coluna existe e os nomes estão corretos nas configurações e no CSV.")
                 self.calculo_df[pont_col_name] = 0
            except Exception as e:
                print(f"  Erro inesperado ao processar coluna '{col}': {type(e).__name__} - {str(e)}")
                import traceback
                # traceback.print_exc() # Descomente para obter traceback completo se necessário
                self.calculo_df[pont_col_name] = 0

        # Calcula pontuação total
        colunas_pontuacao = [c for c in self.calculo_df.columns if c.endswith('_pontuacao')]
        if not colunas_pontuacao:
             print("\nNenhuma coluna de pontuação foi gerada. Pontuação total não calculada.")
             self.calculo_df['Pontuacao_Total'] = 0
        else:
             print(f"\nColunas usadas para Pontuação Total: {colunas_pontuacao}")
             self.calculo_df['Pontuacao_Total'] = self.calculo_df[colunas_pontuacao].sum(axis=1)

        print("\nResumo final das pontuações:")
        # Seleciona colunas de forma segura
        cols_to_show = [self.calculo_df.columns[0]] + colunas_pontuacao + ['Pontuacao_Total']
        valid_cols_to_show = [c for c in cols_to_show if c in self.calculo_df.columns]
        print(self.calculo_df[valid_cols_to_show].head().to_string(index=False))


  # Mostra as pontuações em forma de código (debug apenas)
    def mostrar_pontuacoes(self, apenas_pontuacoes=True):
      """Exibe o DataFrame com as pontuações calculadas

      Args:
          apenas_pontuacoes (bool): Se True, mostra apenas colunas de pontuação
                                    Se False, mostra todas as colunas
      """
      if self.calculo_df is None:
           print("DataFrame de cálculo não disponível.")
           return

      try:
           # Tenta importar display dinamicamente se estiver em um notebook
           from IPython.display import display
      except ImportError:
           # Se não estiver em um notebook, usa print
           display = print

      if apenas_pontuacoes:
          # Filtra apenas colunas que terminam com '_pontuacao' ou são 'Modelo' ou 'Pontuacao_Total'
          # Assume que a primeira coluna é o identificador
          modelo_col = self.calculo_df.columns[0]
          cols_pontuacao = [c for c in self.calculo_df.columns if c.endswith('_pontuacao')]
          cols_total = ['Pontuacao_Total'] if 'Pontuacao_Total' in self.calculo_df.columns else []
          cols = [modelo_col] + cols_pontuacao + cols_total
          valid_cols = [c for c in cols if c in self.calculo_df.columns]
          display(self.calculo_df[valid_cols])
      else:
          display(self.calculo_df)

    def gerar_graficos(self):
        """Gera os gráficos de análise"""
        print("\n--- Gerando Gráficos ---")
        if self.calculo_df is None:
            print("DataFrame de cálculo não disponível para gerar gráficos.")
            return
        try:
            self._gerar_grafico_barras()  # Gráfico de pontuação total
            self._gerar_grafico_numericos()  # Gráficos para cada coluna numérica
            self._gerar_grafico_strings()
            self._gerar_grafico_booleanos()
            print("--- Gráficos Gerados ---")
        except Exception as e:
            print(f"Erro ao gerar gráficos: {e}")
            import traceback
            traceback.print_exc()

    def _gerar_grafico_numericos(self):
        """Gráfico de barras para colunas numéricas com verificação de proporcionalidade"""
        if self.tipos_dados is None:
            print("Tipos de dados não carregados, pulando gráfico de numéricos.")
            return

        # Identifica colunas numéricas
        numeric_cols = [col for col in self.tipos_dados.index
                      if self.tipos_dados[col] == 'number' and col in self.calculo_df.columns]

        if not numeric_cols:
            print("Nenhuma coluna numérica encontrada para gerar gráficos.")
            return

        modelo_col = self.calculo_df.columns[0]  # Assume primeira coluna como identificador

        for col_num in numeric_cols:
            print(f"Gerando gráfico para coluna numérica: '{col_num}'")

            # Prepara dados
            df_num = self.calculo_df[[modelo_col, col_num]].copy()
            df_num = df_num[~df_num[modelo_col].isin(['BOM', 'NEUTRO'])]
            df_num.dropna(subset=[col_num], inplace=True)

            if df_num.empty:
                print(f"Nenhum dado válido para plotar no gráfico de '{col_num}'.")
                continue

            # Obtém valores de referência e tipo de proporcionalidade
            valor_bom = self.valores_bom.get(col_num)
            valor_neutro = self.valores_neutro.get(col_num)
            proporcionalidade = self.proporcionalidade.get(col_num, '')

            # Verifica se é inversamente proporcional
            inversamente_proporcional = proporcionalidade.lower() == 'i_proportional'

            # Adiciona informação ao título
            titulo = f"{col_num} {'(Inversamente Proporcional)' if inversamente_proporcional else ''}"

            # Classifica como positivo/negativo considerando a proporcionalidade
            df_num['Color_Category'] = df_num.apply(
                lambda x: self._classificar_valor(x[col_num], valor_bom, valor_neutro, inversamente_proporcional),
                axis=1
            )

            # Define o mapa de cores
            color_map = {
                'Positiva': 'lightgreen',
                'Negativa': 'palevioletred',
                'Neutra': 'lightgray'
            }

            # Ordena por valor (maior no topo)
            df_num_sorted = df_num.sort_values(col_num, ascending=not inversamente_proporcional)

            # Cria gráfico
            fig = px.bar(df_num_sorted,
                        x=col_num,
                        y=modelo_col,
                        orientation='h',
                        color='Color_Category',
                        color_discrete_map=color_map,
                        hover_name=modelo_col,
                        hover_data={col_num: ':.2f'},
                        title=titulo)

            # Configurações de layout
            fig.update_layout(
                yaxis={'categoryorder': 'total ascending'},
                plot_bgcolor='white',
                paper_bgcolor='white',
                legend_title_text='Resultado',
                xaxis_title=col_num,
                yaxis_title='Modelo'
            )

            # Adiciona linhas de referência e anotações
            if pd.notna(valor_neutro):
                self._adicionar_referencia(fig, float(valor_neutro), len(df_num_sorted),
                                        "Mínimo Aceitável<br>(NEUTRO)", inversamente_proporcional)

            if pd.notna(valor_bom):
                self._adicionar_referencia(fig, float(valor_bom), len(df_num_sorted),
                                        "Desejável (BOM)", inversamente_proporcional)

            print(f"  Gráfico de {col_num} pronto para exibição.")
            fig.show()

    def _classificar_valor(self, valor, valor_bom, valor_neutro, inversamente_proporcional):
        """Classifica o valor como Positiva, Negativa ou Neutra"""
        if pd.isna(valor_neutro):
            return 'Neutra'

        valor = float(valor)
        valor_neutro = float(valor_neutro)

        if inversamente_proporcional:
            return 'Positiva' if valor <= valor_neutro else 'Negativa'
        else:
            return 'Positiva' if valor >= valor_neutro else 'Negativa'

    def _adicionar_referencia(self, fig, valor, y_pos, texto, inversamente_proporcional):
        """Adiciona linha de referência e anotação ao gráfico"""
        fig.add_vline(x=valor, line_dash='dash', line_color='black')

        fig.add_annotation(
            x=valor,
            y=y_pos - 1,
            xref="x",
            yref="y",
            text=texto,
            showarrow=True,
            font=dict(size=16, color="#000000"),
            align="center",
            arrowhead=2,
            arrowsize=1,
            arrowwidth=2,
            arrowcolor="#636363",
            ax=0,
            ay=-45,
            bordercolor="#c7c7c7",
            borderwidth=2,
            borderpad=4,
            bgcolor='white',
            opacity=1
        )

    def _gerar_grafico_barras(self):
        """Gráfico de barras para pontuação total com linha ideal fixa e indicação do melhor produto."""
        print("Gerando gráfico de barras...")
        # Garante que colunas e pesos existem
        if 'Pontuacao_Total' not in self.calculo_df.columns:
            print("Coluna 'Pontuacao_Total' não encontrada. Pulando gráfico de barras.")
            return
        # Verifica se os pesos foram carregados para calcular o máximo teórico
        if self.pesos is None or self.pesos.empty:
             print("Pesos não definidos. Não é possível calcular pontuação máxima teórica. Usando máximo atingido.")
             # Como fallback, podemos usar o máximo atingido ou um valor fixo como 1.0
             # Vamos definir como 1.0 se os pesos não estiverem disponíveis.
             max_score_teorico = 1.0
        else:
            # Calcula a pontuação máxima teórica (soma dos pesos)
            max_score_teorico = self.pesos.sum()
            print(f"  Pontuação máxima teórica calculada (soma dos pesos): {max_score_teorico:.2f}")

        modelo_col = self.calculo_df.columns[0] # Assume primeira coluna como identificador

        df_analise = self.calculo_df[[modelo_col, 'Pontuacao_Total']].copy()
        # Remove linhas BOM/NEUTRO
        df_analise = df_analise[~df_analise[modelo_col].isin(['BOM', 'NEUTRO'])]

        # Remove linhas com pontuação NaN antes de prosseguir e ordenar
        df_analise.dropna(subset=['Pontuacao_Total'], inplace=True)

        if df_analise.empty:
             print("Nenhum dado válido para plotar no gráfico de barras após remover BOM/NEUTRO/NaN.")
             return

        df_analise['Color_Category'] = df_analise.apply(
            lambda x: ('Positiva' if x['Pontuacao_Total'] >= 0 else 'Negativa'),
            axis=1
        )

        color_map = {
            'Positiva': 'lightgreen',
            'Negativa': 'palevioletred'
        }

        # Ordena por pontuação DESCENDENTE para plotagem e identificação do melhor
        df_analise_sorted = df_analise.sort_values('Pontuacao_Total', ascending=False)

        # Identifica o melhor produto (primeira linha após ordenar)
        # Adiciona verificação caso df_analise_sorted fique vazio após dropna
        if df_analise_sorted.empty:
             print("Nenhum produto restante após limpeza para identificar o melhor.")
             top_product_name = "N/A"
             top_product_score = pd.NA
        else:
             top_product = df_analise_sorted.iloc[0]
             top_product_name = top_product[modelo_col]
             top_product_score = top_product['Pontuacao_Total']
             print(f"  Melhor produto identificado: {top_product_name} (Score: {top_product_score:.2f})")


        fig = px.bar(df_analise_sorted,
                    x='Pontuacao_Total',
                    y=modelo_col, # Modelo no eixo Y
                    orientation='h',
                    color='Color_Category',
                    color_discrete_map=color_map,
                    hover_name=modelo_col,
                    hover_data={'Pontuacao_Total':':.2f'}, # Formata hover
                    title='Pontuação Total dos Modelos')

        # Atualiza layout - Ordena eixo Y pela pontuação (melhor em cima)
        fig.update_layout(
            yaxis={'categoryorder': 'total ascending'},
            plot_bgcolor='white',
            paper_bgcolor='white',
            legend_title_text='Resultado',
            xaxis_title='Pontuação Total',
            yaxis_title='Modelo'
        )

        # Linha de referência no neutro
        fig.add_vline(x=0, line_dash='dash', line_color='black')

        # Linha de referência no desejável
        fig.add_vline(x=max_score_teorico, line_dash='dash', line_color='black')

        # Linha Desejável (BOM)
        fig.add_annotation(
            x=max_score_teorico,
            y=len(df_analise_sorted) - 1,
            xref="x",
            yref="y",
            text="Desejável (BOM)<br>(NEUTRO)",
            showarrow=True,
            font=dict(size=16, color="#000000"),
            align="center",
            arrowhead=2,
            arrowsize=1,
            arrowwidth=2,
            arrowcolor="#636363",
            ax=0,
            ay=-45,
            bordercolor="#c7c7c7",
            borderwidth=2,
            borderpad=4,
            bgcolor='white',
            opacity=1
        )

        # Linha Mínimo Aceitável (NEUTRO)
        fig.add_annotation(
            x=0,
            y=len(df_analise_sorted) - 1,
            xref="x",
            yref="y",
            text="Mínimo Aceitável<br>(NEUTRO)",
            showarrow=True,
            font=dict(size=16, color="#000000"),
            align="center",
            arrowhead=2,
            arrowsize=1,
            arrowwidth=2,
            arrowcolor="#636363",
            ax=0,
            ay=-45,
            bordercolor="#c7c7c7",
            borderwidth=2,
            borderpad=4,
            bgcolor='white',
            opacity=1
        )

        # --- Anotação para Melhor Produto ---
        # Só adiciona a anotação se identificamos um melhor produto válido
        if top_product_name != "N/A" and pd.notna(top_product_score):
            fig.add_annotation(
                    x=top_product_score,    # Posição X na pontuação do melhor
                    y=top_product_name,     # Posição Y no nome do melhor (categoria no eixo y)
                    text="🏆 Melhor Produto", # O texto da anotação
                    showarrow=False,        # SEM seta, como você definiu
                    font=dict(
                        color="#00008B",    # Cor da fonte (Azul Escuro)
                        size=12
                    ),
                    # align="center",         # Remova ou comente esta linha

                    # --- Adições para Fundo e Borda ---
                    bgcolor="white",        # Cor do fundo = branco
                    bordercolor="black",    # Cor da borda = preta
                    borderwidth=1,        # Largura da borda = 1 pixel
                    borderpad=4,          # Espaçamento interno = 4 pixels
                    # --- Ajuste de Posição (sem seta) ---
                    # Usamos xshift/yshift para mover o *texto* em relação ao ponto (x,y)
                    # Ajuste esses valores para posicionar onde desejar (ex: acima e à direita da ponta da barra)
                    xshift=-15,            # Experimente com valores negativos (ajuste conforme necessário)
                    yshift=25
            )

        # As anotações antigas que você colou não são necessárias pois usamos annotation_text nas vlines
        # e adicionamos uma anotação específica para o melhor produto.

        print("  Gráfico de barras pronto para exibição.")
        fig.show()


    def _gerar_grafico_strings(self):
        """Gráfico melhorado para variáveis de string - agora para todas as colunas string"""
        if self.tipos_dados is None:
            print("Tipos de dados não carregados, pulando gráfico de strings.")
            return

        string_cols = [col for col in self.tipos_dados.index
                      if self.tipos_dados.get(col) == 'string' and col in self.calculo_df.columns]

        if not string_cols:
            return

        modelo_col = self.calculo_df.columns[0] # Primeira coluna como identificador

        for col_string in string_cols:
            print(f"Gerando gráfico para string: '{col_string}'")
            pts_col_name = self.string_columns_map.get(col_string)

            if not pts_col_name or pts_col_name not in self.df.columns:
                print(f"Aviso: Coluna de pontos '{pts_col_name}' para '{col_string}' não encontrada. Pulando gráfico.")
                continue

            try:
                # Cria mapeamento de pontos base (String -> Ponto numérico)
                dados_mapa = self.df.iloc[5:].copy()
                dados_mapa[pts_col_name] = pd.to_numeric(dados_mapa[pts_col_name], errors='coerce')
                dados_mapa = dados_mapa.dropna(subset=[col_string, pts_col_name])
                valores_string_base = pd.Series(dados_mapa[pts_col_name].values, index=dados_mapa[col_string]).to_dict()

                if not valores_string_base:
                    print(f"Aviso: Mapeamento de pontos base para '{col_string}' vazio. Pulando gráfico.")
                    continue

            except Exception as e:
                print(f"Erro ao criar mapeamento para gráfico de string '{col_string}': {e}.")
                continue

            df_string = self.calculo_df[[modelo_col, col_string]].copy()
            df_string = df_string[~df_string[modelo_col].isin(['BOM', 'NEUTRO'])]

            if df_string.empty:
                print(f"Nenhum dado para plotar no gráfico de string para '{col_string}'.")
                continue

            # Mapeia para obter o PONTO BASE para o gráfico (não a pontuação final ponderada)
            df_string['ValorBase'] = df_string[col_string].map(valores_string_base).fillna(0)
            df_string = df_string.sort_values('ValorBase', ascending=False)
            df_string['Posicao'] = range(len(df_string)) # Posição X baseada na ordenação

            fig = px.scatter(df_string,
                            x='Posicao',
                            y='ValorBase', # Usa o valor base no eixo Y
                            color=col_string, # Cor baseada na categoria string
                            hover_name=modelo_col, # O que aparece ao passar o mouse
                            title=f'Comparação de {col_string.capitalize()} entre Modelos (Pontuação Base)',
                            size_max=15, # Tamanho dos pontos
                            hover_data={'Posicao': False, 'ValorBase': True, col_string: True},
                            labels={'ValorBase': 'Pontuação Base (do CSV)', col_string: col_string.capitalize()})

            # Adiciona linhas BOM/NEUTRO com base nos valores BASE do mapeamento
            valor_bom_str = self.valores_bom.get(col_string)
            valor_neutro_str = self.valores_neutro.get(col_string)
            valor_base_bom = valores_string_base.get(valor_bom_str, None) # Busca ponto base do BOM
            valor_base_neutro = valores_string_base.get(valor_neutro_str, None) # Busca ponto base do NEUTRO

            if valor_base_bom is not None:
                fig.add_hline(y=valor_base_bom, line_dash='dash',
                          line_color='green', line_width=2,
                          annotation_text=f"BOM: '{valor_bom_str}' ({valor_base_bom:.2f})",
                          annotation_position="top right",
                          annotation_font=dict(color='green', size=12))

            if valor_base_neutro is not None:
                fig.add_hline(y=valor_base_neutro, line_dash='dash',
                          line_color='orange', line_width=2,
                          annotation_text=f"NEUTRO: '{valor_neutro_str}' ({valor_base_neutro:.2f})",
                          annotation_position="bottom right",
                          annotation_font=dict(color='orange', size=12))

            fig.update_layout(
                xaxis=dict(title='Modelos Ordenados por Pontuação Base', showticklabels=False, showgrid=False, zeroline=False),
                yaxis=dict(title='Pontuação Base (Definida no CSV)', showgrid=True, gridcolor='lightgray', zeroline=True, zerolinecolor='lightgray'),
                plot_bgcolor='white', paper_bgcolor='white',
                margin=dict(l=40, r=40, t=60, b=40),
                legend=dict(title_text=col_string.capitalize(), orientation='h', yanchor='bottom', y=-0.3, xanchor='center', x=0.5),
                hoverlabel=dict(bgcolor='white', font_size=12, font_family='Arial')
            )

            # CORREÇÃO: Removido o parâmetro 'selector' que causava o erro
            fig.update_traces(marker=dict(size=12, line=dict(width=1, color='DarkSlateGrey'), opacity=0.8))

            fig.show()

    def _gerar_grafico_booleanos(self):
        """Gráfico para todas as variáveis booleanas"""
        # Identifica colunas booleanas
        bool_cols = [col for col in self.tipos_dados.index
                    if self.tipos_dados[col] == 'boolean' and col in self.calculo_df.columns]

        if not bool_cols:
            return

        modelo_col = self.calculo_df.columns[0]

        for col_bool in bool_cols:
            df_bool = self.calculo_df[~self.calculo_df[modelo_col].isin(['BOM', 'NEUTRO'])][[modelo_col, col_bool]].copy()
            df_bool['Valor'] = df_bool[col_bool].map({True: 1, False: 0})
            df_bool['Tamanho'] = 20

            fig = px.scatter(df_bool,
                    x=modelo_col,
                    y='Valor',
                    color=col_bool,
                    color_discrete_map={True: '#4CAF50', False: '#F44336'},
                    size='Tamanho',
                    title=f'{col_bool} por Modelo',
                    hover_name=modelo_col,
                    hover_data={'Valor': False, 'Tamanho': False},
                    category_orders={col_bool: [True, False]})

            # Atualiza a legenda para mostrar SIM/NÃO
            fig.for_each_trace(lambda t: t.update(name='SIM' if t.name == 'True' else 'NÃO'))

            fig.update_yaxes(
                range=[-0.5, 1.5],
                tickvals=[0, 1],
                ticktext=['Não', 'Sim'],
                showgrid=False
            )

            # Adiciona linhas de referência SIM/NÃO
            fig.add_hline(y=1, line_dash='dash', line_color='green')
            fig.add_hline(y=0, line_dash='dash', line_color='orange')

            # Altera aparência do gráfico
            fig.update_layout(
                xaxis_title=None,
                yaxis_title=None,
                plot_bgcolor='white',
                paper_bgcolor='white'
            )

            # Linhas de referência
            valor_bom = 1 if self.valores_bom[col_bool] else 0
            valor_neutro = 1 if self.valores_neutro[col_bool] else 0

            fig.show()

    def recomendar_produtos(self):
        """Recomenda produtos com base na pontuação incluindo todos os critérios"""
        # Assume que a primeira coluna é o identificador 'Modelo'
        modelo_col = self.df.columns[0]
        if 'Pontuacao_Total' not in self.calculo_df.columns:
            print("Pontuação total não calculada. Não é possível recomendar.")
            return

        produtos = self.calculo_df[~self.calculo_df[modelo_col].isin(['BOM', 'NEUTRO'])]

        if produtos.empty:
            print("Nenhum produto encontrado para recomendação (após excluir BOM/NEUTRO).")
            return

        # Verifica se há pontuações válidas
        if produtos['Pontuacao_Total'].isna().all():
            print("Todas as pontuações totais são inválidas. Não é possível recomendar.")
            return

        pontuacao_maxima = produtos['Pontuacao_Total'].max()
        # Lida com o caso de todas as pontuações serem NaN ou não haver máximo
        if pd.isna(pontuacao_maxima):
             print("Não foi possível determinar a pontuação máxima válida.")
             return

        recomendados = produtos[produtos['Pontuacao_Total'] == pontuacao_maxima]

        print("\n--- Produtos Recomendados ---")
        if recomendados.empty:
             print("Nenhum produto atingiu a pontuação máxima (pode indicar erro ou dados uniformes).")
             return

        for _, produto in recomendados.iterrows():
            print(f"\nProduto: {produto[modelo_col]} com pontuação total de {produto['Pontuacao_Total']:.2f}")
            print("Detalhes da Pontuação por Critério (Ordenado por Peso):")

            # Ordena critérios por peso (do maior para o menor)
            # Garante que o peso existe antes de tentar ordenar
            criterios_ordenados = sorted(
                [(col, self.pesos[col]) for col in self.pesos.index if col in self.pesos],
                key=lambda item: item[1], reverse=True
            )

            for col, peso in criterios_ordenados:
                pont_col = f"{col}_pontuacao"
                # Verifica se a coluna de pontuação existe no produto e tem valor válido
                if pont_col not in produto or pd.isna(produto[pont_col]):
                    # print(f"  - {col}: Pontuação não disponível ou inválida.")
                    continue # Pula se não houver pontuação para este critério

                pontuacao = produto[pont_col]
                valor_atual = produto.get(col, "N/A") # Pega o valor original com segurança
                tipo = self.tipos_dados.get(col, "desconhecido")

                # Formata a saída para melhor leitura
                justificativa = f"(Peso: {peso:.2f}, Pontos: {pontuacao:.2f})"

                if tipo == 'string':
                    valor_atual_str = f"'{valor_atual}'" if valor_atual != "N/A" else valor_atual
                    if pontuacao > 0:
                        print(f"  - {col}: {valor_atual_str} [Vantagem] {justificativa}")
                    elif pontuacao < 0:
                         print(f"  - {col}: {valor_atual_str} [Desvantagem] {justificativa}")
                    else:
                         print(f"  - {col}: {valor_atual_str} [Neutro] {justificativa}")

                elif tipo == 'boolean':
                     valor_atual_str = "Sim" if valor_atual == True else ("Não" if valor_atual == False else "N/A")
                     if pontuacao > 0:
                          print(f"  - {col}: {valor_atual_str} [Vantagem] {justificativa}")
                     elif pontuacao < 0:
                          print(f"  - {col}: {valor_atual_str} [Desvantagem] {justificativa}")
                     else:
                          print(f"  - {col}: {valor_atual_str} [Neutro] {justificativa}")

                elif tipo == 'number':
                     valor_atual_str = f"{valor_atual:.1f}" if isinstance(valor_atual, (int, float)) else str(valor_atual)
                     valor_bom = self.valores_bom.get(col)
                     valor_neutro = self.valores_neutro.get(col)

                     desc = f"{valor_atual_str}"
                     if pd.notna(valor_bom) and pd.notna(valor_neutro):
                          desc += f" (Bom: {valor_bom:.1f}, Neutro: {valor_neutro:.1f})"

                     if pontuacao >= peso * 0.99: # Praticamente atingiu o máximo
                          print(f"  - {col}: {desc} [Ótimo] {justificativa}")
                     elif pontuacao > 0:
                          print(f"  - {col}: {desc} [Bom] {justificativa}")
                     elif pontuacao == 0:
                           print(f"  - {col}: {desc} [Neutro] {justificativa}")
                     else: # pontuacao < 0
                          print(f"  - {col}: {desc} [Abaixo do Neutro] {justificativa}")
                else:
                     # Caso tipo desconhecido
                     print(f"  - {col}: {valor_atual} [Tipo Desconhecido] {justificativa}")


# # --- Exemplo de Uso - Notebook ---
# # Certifique-se que o arquivo 'base_eletronicos_2.csv' está no mesmo diretório
# # ou forneça o caminho completo.
# try:
#     print("--- Análise para base de eletrônicos ---")
#     analyzer_eletro = DataAnalyzer('base_eletronicos_2.csv')
#     analyzer_eletro.load_and_prepare_data()
#     analyzer_eletro.calcular_pontuacoes()
#     analyzer_eletro.gerar_graficos()
#     analyzer_eletro.recomendar_produtos()
# except FileNotFoundError:
#     print("\nErro: Arquivo 'base_eletronicos_2.csv' não encontrado.")
# except Exception as e:
#     print(f"\nOcorreu um erro inesperado durante a análise: {e}")
#     import traceback
#     traceback.print_exc()

# --- Exemplo de Uso - Carro ---
# Certifique-se que o arquivo 'base_eletronicos_2.csv' está no mesmo diretório
# ou forneça o caminho completo.

try:
    print("--- Análise para base de carros ---")
    analyzer_carro = DataAnalyzer('base_carro.csv')
    analyzer_carro.load_and_prepare_data()
    analyzer_carro.calcular_pontuacoes()
    analyzer_carro.gerar_graficos()
    analyzer_carro.recomendar_produtos()
except FileNotFoundError:
    print("\nErro: Arquivo 'base_carro.csv' não encontrado.")
except Exception as e:
    print(f"\nOcorreu um erro inesperado durante a análise: {e}")
    import traceback
    traceback.print_exc()

--- Análise para base de carros ---
Coluna string 'Cor' mapeada para 'Cor,pts_string' (original: 'Unnamed: 9')
Colunas disponíveis para cálculo: ['Modelo', 'Preço (R$)', 'Consumo (km/l)', 'Potência (cv)', 'Porta-malas (litros)', 'Segurança (estrelas)', 'Ar Condicionado', 'Câmbio Automático', 'Cor', 'Cor,pts_string']
Pesos aplicáveis: {'Preço (R$)': 0.3, 'Consumo (km/l)': 0.1, 'Potência (cv)': 0.1, 'Porta-malas (litros)': 0.05, 'Segurança (estrelas)': 0.2, 'Ar Condicionado': 0.1, 'Câmbio Automático': 0.1, 'Cor': 0.05}
Tipos de Dados: {'Preço (R$)': 'number', 'Consumo (km/l)': 'number', 'Potência (cv)': 'number', 'Porta-malas (litros)': 'number', 'Segurança (estrelas)': 'number', 'Ar Condicionado': 'boolean', 'Câmbio Automático': 'boolean', 'Cor': 'string'}

Processando coluna: Preço (R$)
  Tipo: number, Proporcionalidade: i_proportional
  Valores BOM: 60000, NEUTRO: 85000, Peso: 0.3
  Resultado parcial (Top 5):
 Preço (R$)  Preço (R$)_pontuacao
      75000                 0.120
      82

Gerando gráfico para coluna numérica: 'Preço (R$)'
  Gráfico de Preço (R$) pronto para exibição.


Gerando gráfico para coluna numérica: 'Consumo (km/l)'
  Gráfico de Consumo (km/l) pronto para exibição.


Gerando gráfico para coluna numérica: 'Potência (cv)'
  Gráfico de Potência (cv) pronto para exibição.


Gerando gráfico para coluna numérica: 'Porta-malas (litros)'
  Gráfico de Porta-malas (litros) pronto para exibição.


Gerando gráfico para coluna numérica: 'Segurança (estrelas)'
  Gráfico de Segurança (estrelas) pronto para exibição.


Gerando gráfico para string: 'Cor'


--- Gráficos Gerados ---

--- Produtos Recomendados ---

Produto: Honda City com pontuação total de 0.56
Detalhes da Pontuação por Critério (Ordenado por Peso):
  - Preço (R$): 92000.0 (Bom: 60000.0, Neutro: 85000.0) [Abaixo do Neutro] (Peso: 0.30, Pontos: -0.02)
  - Segurança (estrelas): 5.0 (Bom: 5.0, Neutro: 3.0) [Ótimo] (Peso: 0.20, Pontos: 0.20)
  - Consumo (km/l): 11.8 (Bom: 15.0, Neutro: 10.0) [Bom] (Peso: 0.10, Pontos: 0.04)
  - Potência (cv): 126.0 (Bom: 120.0, Neutro: 90.0) [Ótimo] (Peso: 0.10, Pontos: 0.10)
  - Ar Condicionado: Sim [Vantagem] (Peso: 0.10, Pontos: 0.10)
  - Câmbio Automático: Sim [Vantagem] (Peso: 0.10, Pontos: 0.10)
  - Porta-malas (litros): 500.0 (Bom: 400.0, Neutro: 300.0) [Ótimo] (Peso: 0.05, Pontos: 0.05)
  - Cor: 'Azul' [Vantagem] (Peso: 0.05, Pontos: 0.00)
